In [1]:
# Face Emotion Detection of image, video, webcam

import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model('full.h5')

# Load Haar cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Emotion labels (change if your model uses different order)
emotion_labels = ['angry', 'disgust', 'fear', 'happy', 'surprise', 'sad', 'neutral']


# Model input size
target_size = (224, 224)

def detect_and_predict(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

    for (x, y, w, h) in faces:
        roi_color = frame[y:y+h, x:x+w]
        try:
            roi_color = cv2.resize(roi_color, target_size)
        except:
            continue  # skip if face too small

        roi = roi_color.astype('float32') / 255.0
        roi = np.expand_dims(roi, axis=0)  # shape becomes (1, 224, 224, 3)

        prediction = model.predict(roi, verbose=0)
        label = emotion_labels[np.argmax(prediction)]

        # Draw rectangle and label
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, label, (x, y-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    return frame

def process_image(image_path):
    frame = cv2.imread(image_path)
    if frame is None:
        print(f"Cannot load image: {image_path}")
        return
    frame = detect_and_predict(frame)
    cv2.imshow('Face Emotion Detection - Image', frame)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def process_video(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Cannot open video: {video_path}")
        return
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = detect_and_predict(frame)
        cv2.imshow('Face Emotion Detection - Video', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

def process_webcam():
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Cannot open webcam")
        return
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame")
            break
        frame = detect_and_predict(frame)
        cv2.imshow('Face Emotion Detection - Webcam', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    print("Select mode:")
    print("1: Detect emotion on an image")
    print("2: Detect emotion on a video")
    print("3: Detect emotion from webcam")
    choice = input("Enter choice (1/2/3): ")

    if choice == '1':
        path = input("Enter image file path: ")
        process_image(path)
    elif choice == '2':
        path = input("Enter video file path: ")
        process_video(path)
    elif choice == '3':
        process_webcam()
    else:
        print("Invalid choice.")


Select mode:
1: Detect emotion on an image
2: Detect emotion on a video
3: Detect emotion from webcam


Enter choice (1/2/3):  3


IndexError: list index out of range